Sonia DAKHLI p1905223

Kévin TANG p1501263

# TP Troll

UE Théorie des Jeux

Enseignant : Gérald GAVIN 

Année 2022-2023

In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 29.9 MB/s eta 0:00:00


In [ ]:
NB_CASES = 7 # incluent les 2 château
NB_PIERRES = 15
NB_PIERRES_J1 = 16
NB_PIERRES_J2 = 16
POS_INITIALE = 0

In [ ]:
#Code du prof
import random
import numpy as np
import pulp
from sympy import *

def get_nash(J):
        lines=range(len(J))
        columns=range(len(J[0]))

        lp = pulp.LpProblem('Nash', pulp.LpMaximize)
        V = pulp.LpVariable('V', cat='Continuous')

        a = pulp.LpVariable.dicts("a", (lines),  cat="Continous")
        lp.setObjective(V)

        lp += pulp.lpSum([a[i] for i in lines]) == 1

        for i in lines :
            lp +=a[i]>=0

        for j in columns :
            lp += pulp.lpSum([a[i]*J[i][j] for i in lines])-V>=0


        lp.solve()

        return [pulp.value(lp.objective), [pulp.value(a[i]) for i in lines]]

m=4
J=np.zeros((m)*(m))
J.resize((m, m))
for i in range(m):
        for j in range(m):
            if i==j :
                J[i][j]=5
            else:
                J[i][j]=-abs((i-j))

qq=get_nash(J)
print(qq)


[-0.041666667, [0.29166667, 0.20833333, 0.20833333, 0.29166667]]




---



In [ ]:
def decaleIndiceDesc(i):
    i -= int(NB_CASES/2)
    return i

In [ ]:
def decaleIndice(i):
    i += int(NB_CASES/2)
    return i

In [ ]:
# Initialisation de la matrice J stockant les valeurs des gains aux indices :
# x : nombre de pierres du joueur 1
# y : nombre de pierres du joueur 2
# z : nombre de cases (incluant les deux châteaux)
J=np.full((NB_PIERRES_J1 * NB_PIERRES_J2 * NB_CASES), None)
J.resize((NB_PIERRES_J1, NB_PIERRES_J2, NB_CASES))
# Initialisation de la matrice J stockant les valeurs des sous-jeux :
# x : nombre de pierres du joueur 1
# y : nombre de pierres du joueur 2
# z : nombre de cases (incluant les deux châteaux)
SubGames = [[[[] for t in range(NB_CASES)] for y in range(NB_PIERRES_J2)] for x in range(NB_PIERRES_J1)]

# Remplit les Matrices J(x, 0, t), J(0, y, t) et J(x, x, 0)
def initJeu():
    # Matrice J(x, 0, t)
    for x in range(NB_PIERRES_J1):
        for t in range(NB_CASES):
            t2 = decaleIndiceDesc(t) # permet d'obtenir l'indice de -3 à 3 (dans le cas 7) au lieu de 0 à 6
            if t==0: # on a atteint le château 1 => le joueur 1 a perdu
                J[x][0][t] = -1
            elif t==NB_CASES-1: # on a atteint le château 2 => le joueur 2 a perdu
                J[x][0][t] = 1
            elif x == abs(t2): # le troll est en position 0 => jeu nul
                J[x][0][t] = 0
            elif x > t2: # le troll est plus proche du château du joueur 2 => le joueur 1 a gagné
                J[x][0][t] = 1
            elif x < t2: # le troll est plus proche du château du joueur 1 => le joueur 1 a perdu
                J[x][0][t] = -1
                
                
    # Matrice J(0, y, t)
    for y in range(NB_PIERRES_J2):
        for t in range(NB_CASES):
            t2 = decaleIndiceDesc(t) 
            if t==0: # on a atteint le château 1 => le joueur 1 a perdu
                J[0][y][t] = -1
            elif t==NB_CASES-1: # on a atteint le château 2 => le joueur 2 a perdu
                J[0][y][t] = 1
            elif y == t2: # le troll est en position 0 => jeu nul
                J[0][y][t] = 0
            elif y < t2: # le troll est plus proche du château du joueur 1 => le joueur 1 a perdu
                J[0][y][t] = 1
            elif y > t2: # le troll est plus proche du château du joueur 2 => le joueur 1 a gagné
                J[0][y][t] = -1
                
    # Matrice J(x, x, 0)
    for x in range(max(NB_PIERRES_J1, NB_PIERRES_J2)):
        J[x][x][decaleIndice(0)] = 0
         
initJeu()   



In [ ]:
# Remplit la matrice J avec ses valeurs pour tous les x y et t
def fillMatrice():
  for x in range(1, NB_PIERRES_J1):
    for y in range(1, NB_PIERRES_J2):
      for t in range(NB_CASES):
        SJ = np.zeros((x)*(y))
        SJ.resize((x, y))

        for a in range(x):
          for b in range(y):
            # Calcule la nouvelle position du troll
            t2 = 0
            if (x-a)>(y-b):
              t2 = 1
            elif (x-a)<(y-b):
              t2 = -1
            tnew = t + t2
            
            # Trouve la valeur a la case a, b, tnew du sous-jeu
            if (tnew < NB_CASES-1 and tnew > 0):
              SJ[a][b] = J[a][b][tnew]
            elif (tnew >= NB_CASES-1): # Si le troll a atteint le château 2 => victoire du joueur 1
              SJ[a][b] = 1
            elif (tnew <= 0): # Si le troll a atteint le château 1 => victoire du joueur 2
              SJ[a][b] = -1
        # Inverse le tableau 2D
        SJ = SJ[::-1, ::-1]
        result = get_nash(SJ)
        J[x][y][t] = result[0]
        SubGames[x][y][t] = SJ

fillMatrice()

In [ ]:
# Calcule l'équilibre de Nash d'une matrice 3d
# J : matrice 3d
# x : valeur de x
# y : valeur de y
# t : valeur de t
def get_nash_3d(J, x, y, t):
    SJ = np.zeros(x * y)
    SJ.resize(x, y)
    for a in range(x):
        for b in range(y):
            t2 = 0
            if (x-a)>(y-b):
                t2+=1
            elif (x-a)<(y-b):
                t2-=1
            if (t+t2 < NB_CASES-1 and t+t2>0):
                SJ[a][b] = J[a][b][t+t2]
            elif (t+t2 >= NB_CASES-1):
                SJ[a][b] = 1
            elif (t+t2 <= 0):
                SJ[a][b] = -1
    
    # Inverse le tableau 2D
    SJ = SJ[::-1, ::-1]

    return get_nash(SJ)

In [ ]:
get_nash(SubGames[15][15][decaleIndice(0)])

[0.0,
 [0.0,
  0.46122431,
  0.048835351,
  0.48994034,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]]

In [ ]:
print(J[15][15][decaleIndice(0)])
get_nash_3d(J, 15, 15, decaleIndice(0))

0.0


[0.0,
 [0.0,
  0.46122431,
  0.048835351,
  0.48994034,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]]



---



In [ ]:
class Joueur:

  def __init__(self, nom, nb_pierres, position):
        self.name = nom
        self.nb_pierres = nb_pierres
        self.position = position

  def getName(self):
        return self.name
  
  def getNbPierres(self):
        return self.nb_pierres
  
  def jettePierres(self, n):
        if n > self.nb_pierres:
            self.nb_pierres = 0
        else:
            self.nb_pierres -= n
  
  def aZeroPierres(self):
        return self.nb_pierres <= 0
  
  def getPosition(self):
        return self.position


In [ ]:
class Troll:
  def __init__(self, position):
        self.position = position
  
  def getPosition(self):
        return self.position

  def moveTroll(self, n1, n2):
        if n1 > n2:
            self.position += 1
        elif n1 < n2:
            self.position -= 1

  def moveTrollEnd(self, j1, j2, n):
        if j1 > j2:
          self.position += n
        elif j1 < j2:
          self.position -= n



In [ ]:
class Game:
  def __init__(self, player1, troll, player2):
        self.player1 = player1
        self.troll = troll
        self.player2 = player2
  
  def isOver(self):
        # le troll a atteint le chateau du joueur 1
        if self.troll.getPosition() == self.player1.getPosition():
            print("Le Troll à atteint le chateau de " + self.player1.getName())
            return True

        # le troll a atteint le chateau du joueur 2
        if self.troll.getPosition() == self.player2.getPosition():
            print("Le Troll à atteint le chateau de " + self.player2.getName())
            return True

        # les 2 joueurs n'ont plus aucune pierre à jeter
        if self.player1.aZeroPierres() and self.player2.aZeroPierres():
            
            # on regarde ou est la position du troll
            if self.troll.getPosition() == 0:
                print("Les 2 joueurs n'ont plus aucune pierre à jeter: le Troll est au milieu. Egalité parfaite")
            elif self.troll.getPosition() < 0:
                print("Les 2 joueurs n'ont plus aucune pierre à jeter. Le Troll est proche du chateau de " + self.player1.getName())
            else:
                print("Les 2 joueurs n'ont plus aucune pierre à jeter. Le Troll est proche du chateau de " + self.player2.getName())

            return True

        return False

  
  def startRound(self):
        # on vérifie si l'un des 2 joueurs n'a plus de pierre: si oui, le jeu se termine automatiquement
        if self.player1.aZeroPierres():
            # le joueur 1 n'a plus de pierres
            stone1 = 0
            stone2 = self.player2.getNbPierres()
            self.player2.jettePierres(stone2)
            self.troll.moveTrollEnd(stone1, stone2, stone2)
            print(self.player1.getName() + " n'a plus de pierres: " + self.player2.getName() + " va donc jeter ses pierres")
            stone1 = 0
            stone2 = 0
        elif self.player2.aZeroPierres():
            # le joueur 2 n'a plus de pierres
            stone1 = self.player1.getNbPierres()
            stone2 = 0
            self.player1.jettePierres(stone1)
            self.troll.moveTrollEnd(stone1, stone2, stone1)
            print(self.player2.getName() + " n'a plus de pierres: " + self.player1.getName() + " va donc jeter ses pierres")
            stone1 = 0
            stone2 = 0
        else:
            # chacun des joueurs choisit sa stratégie

            # Partie a modif pour les stratégies
            sj1 = SubGames[self.player1.getNbPierres()][self.player2.getNbPierres()][decaleIndice(self.troll.getPosition())]
            qq_J1 = get_nash(sj1)
            stone1 = qq_J1[1].index(max(qq_J1[1])) + 1

            sj2 = SubGames[self.player2.getNbPierres()][self.player1.getNbPierres()][decaleIndice(self.troll.getPosition())]
            qq_J2 = get_nash(sj2)
            stone2 = qq_J2[1].index(max(qq_J2[1])) + 1

            print(self.player1.getName() + " jette " + str(stone1) + " pierres contre " + self.player2.getName() + " qui jette " + str(stone2) + " pierres")
             
        self.player1.jettePierres(stone1)
        self.player2.jettePierres(stone2)
        self.troll.moveTroll(stone1, stone2)
  
  def stats(self):
        return [self.player1.getNbPierres(), self.player2.getNbPierres(), self.troll.getPosition()]


In [ ]:
#nb de pierres
NB_PIERRES = 15

#nb de cases
NB_CASES = 7 

POSITION_TROLL = 0

def main(): 
  # création des joueurs
  p1 = Joueur("Joueur 1", NB_PIERRES, -(NB_CASES // 2))
  p2 = Joueur("Joueur 2", NB_PIERRES, (NB_CASES // 2))

  # création du troll
  troll = Troll(POSITION_TROLL)

  # création de la partie
  game = Game(p1, troll, p2)

  #Affichage du statut du jeu
  print(game.stats())

  #Boucle d'exécution du jeu
  while not game.isOver():
        game.startRound()
        print(game.stats())



In [ ]:
#Executer le main
main()

[15, 15, 0]
Joueur 1 jette 4 pierres contre Joueur 2 qui jette 4 pierres
[11, 11, 0]
Joueur 1 jette 2 pierres contre Joueur 2 qui jette 2 pierres
[9, 9, 0]
Joueur 1 jette 2 pierres contre Joueur 2 qui jette 2 pierres
[7, 7, 0]
Joueur 1 jette 2 pierres contre Joueur 2 qui jette 2 pierres
[5, 5, 0]
Joueur 1 jette 1 pierres contre Joueur 2 qui jette 1 pierres
[4, 4, 0]
Joueur 1 jette 1 pierres contre Joueur 2 qui jette 1 pierres
[3, 3, 0]
Joueur 1 jette 1 pierres contre Joueur 2 qui jette 1 pierres
[2, 2, 0]
Joueur 1 jette 1 pierres contre Joueur 2 qui jette 1 pierres
[1, 1, 0]
Joueur 1 jette 1 pierres contre Joueur 2 qui jette 1 pierres
[0, 0, 0]
Les 2 joueurs n'ont plus aucune pierre à jeter: le Troll est au milieu. Egalité parfaite
